# M161 first question notebook- best model LogisticRegrgessionCV , 0.9929 accuracy, 111220 (all) instances, max_features=35000 vectorization, n_gram (1,2)

### Data from D:\Github\bigData\part1\joblibCache\dataTrain_cleaned.joblib
(duplicate removal and text processed already including stemming an d lemmatize)

In [1]:
import joblib

dataTrain = joblib.load(r'joblibCache\dataTrain_cleaned_onlyLematize.joblib')
dataTrain.head()

,Id,Title,Content,Label
0,227464,coming cable grocery overlord,subscribe one three dink comparatively speakin...,Entertainment
1,244074,president react happy,president react happy singer president took tw...,Entertainment
2,60707,wildlife service,fish wildlife service comment period additiona...,Technology
3,27883,launch,nature social medium often source real time br...,Technology
4,169596,u new york casino,u new york casino latest news top deck world e...,Business


In [ ]:
# # Stratify and keep 50000 instances based on the 'Label' column
# from sklearn.model_selection import train_test_split

# # Stratify and sample 50000 instances
# stratified_data, _ = train_test_split(
#     dataTrain,
#     train_size=50000,
#     stratify=dataTrain['Label'],
#     random_state=42
# )
# dataTrain = stratified_data.reset_index(drop=True)
# print(f"Subset shape (stratified): {dataTrain.shape}")

### Just printing out the firtst 5 columns to see what happend to text

In [2]:
print(dataTrain.info())

<class 'pandas.DataFrame'>
RangeIndex: 111220 entries, 0 to 111219
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   Id       111220 non-null  int64
 1   Title    111220 non-null  str  
 2   Content  111220 non-null  str  
 3   Label    111220 non-null  str  
dtypes: int64(1), str(3)
memory usage: 3.4 MB
None


## Starting future extraction (converting text to numbers for ML algorythms to run)





### TF-IDF Vectorization

We will now use TF-IDF vectorization instead of Bag of Words to represent the text data for classification. TF-IDF often improves performance by reducing the impact of common words and highlighting more informative terms.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine Title and Content if not already done
if 'Combined' not in dataTrain.columns:
    dataTrain['Combined'] = dataTrain['Title'].fillna('') + ' ' + dataTrain['Content'].fillna('')

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=100000)
dataTrain_tfidf = vectorizer.fit_transform(dataTrain['Combined'])

print('TF-IDF matrix shape:', dataTrain_tfidf.shape)
print('Feature names (first 20):', vectorizer.get_feature_names_out()[:20])

TF-IDF matrix shape: (111220, 100000)
Feature names (first 20): ['aa' 'aa betty' 'aal' 'aam' 'aba' 'aback' 'abandon' 'abandoned'
 'abandoned king' 'abandonment' 'abatement' 'abbas' 'abbey' 'abdomen'
 'abdominal' 'abdominal obesity' 'abdominal pain' 'abduction' 'abed'
 'aberration']


In [5]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score
import numpy as np

# 1. Keep data sparse! Avoid .toarray() to save memory
X = dataTrain_tfidf 
y = dataTrain['Label'].values

# 2. Scale the data globally before the CV loop
# MaxAbsScaler is ideal for TF-IDF as it scales features to [-1, 1] 
# without removing the zeros (preserving sparsity).
scaler = MaxAbsScaler()
# scaler = StandardScaler(with_mean=False)  # StandardScaler with with_mean=False to preserve sparsity

X_scaled = scaler.fit_transform(X)

# 3. Initialize CV and Model
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_acc = []

# Use a standard SGDClassifier with 'early_stopping' to prevent overfitting
sgd = SGDClassifier(
    alpha=0.00001,           # Regularization strength
    loss='hinge',           # Linear SVM
    penalty='l2',           # Standard regularization
    max_iter=10000,          # Maximum passes over the data
    tol=1e-3,               # Stopping criterion
    n_jobs=4,              
    random_state=42,
    learning_rate='adaptive',  # Adjust learning rate based on performance
    warm_start=True           # Reuse the solution of the previous fit as initialization
)

# 4. The CV Loop
print("Starting Cross-Validation...")
for i, (train_idx, test_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Standard fit (handles internal iterations automatically)
    sgd.fit(X_train, y_train)
    
    # Evaluation
    pred = sgd.predict(X_test)
    acc = accuracy_score(y_test, pred)
    cv_acc.append(acc)
    print(f"Fold {i+1} Accuracy: {acc:.4f}")

print("-" * 30)
print(f'Mean CV Accuracy: {np.mean(cv_acc):.4f}')
print(f'Standard Deviation: {np.std(cv_acc):.4f}')

Starting Cross-Validation...
Fold 1 Accuracy: 0.9631
Fold 2 Accuracy: 0.9938
Fold 3 Accuracy: 0.9965
Fold 4 Accuracy: 0.9968
Fold 5 Accuracy: 0.9974
------------------------------
Mean CV Accuracy: 0.9895
Standard Deviation: 0.0132
